In [2]:
###### 임포트
import pandas as pd
import numpy as np
import joblib
import json

###### 로드 및 추출
df = pd.read_pickle(rf'C:/Users/401-14/Desktop/30-200/mel_genre/extracted_dataframe.pickle') # CNN 특징추출 (장르)
loaded_model = joblib.load(rf'C:/Users/401-14/Desktop/30-200/mel_genre/knn_model.joblib')    # KNN 이웃추천
X = np.array( [np.array(x) for x in df['total']] )           # Data 정렬

###### 함수정의

# song_id -> index
def get_indices_from_song_ids(song_ids):
    indices = []
    for song_id in song_ids:
        try:
            index = df[df['key'] == song_id].index.values[0]
            indices.append(index)
        except IndexError:
            pass
    return indices

# find random neighbor indices (recommend neighbor caculated by mel_spectrum with genre)
def get_random_neighbor_indices(index_list, k=10, random_samples=5):
    random_neighbors = set()
    for index in index_list:
        query_point = X[index].reshape(1, -1)
        indices = loaded_model.kneighbors(query_point, n_neighbors=k, return_distance=False)
        random_neighbors.update(indices[0][1:])
    
    # Convert the set to a list and shuffle
    random_neighbors_list = list(random_neighbors)
    np.random.shuffle(random_neighbors_list)
    
    # Take random samples (default: 5)
    random_neighbors_list = random_neighbors_list[:random_samples]
    
    return random_neighbors_list

# convert list of indices to song_id values
def indices_to_song_id(data_frame, indices_list):
    song_id_list = [data_frame.loc[index, 'key'] for index in indices_list]
    return song_id_list

def get_data_by_keys(df, random_neighbors):
    data_list = []
    for index, row in df.iterrows():
        # Check if the 'key' column value is
        if row['key'] in random_neighbors:
            # Create a dictionary
            data_dict = {
                'key': row['key'], ##### 만약에 key를 song_id로 전달해줘야하면 수정해야할 부분.
                'song_name': row['song_name'],
                'artist_name': row['artist_name'],
                'lyric_str': row['lyric_str'],
                'url_new_song': row['url_new_song']
            }

            # Add the dictionary to the list
            data_list.append(data_dict)
            # Convert the list of dictionaries to a JSON string
            json_data = json.dumps(data_list)
            
    return data_list ##### json_data(json_dump를 해야할 경우)

# 전체 순차 실행 함수
def run(index_list):
    input_to_function = get_indices_from_song_ids(index_list)
    # print("Input to Function:", input_to_function)
    random_neighbors = get_random_neighbor_indices(input_to_function, k=10)
    # print("Random Neighbor Indices:", random_neighbors)
    song_id_list = indices_to_song_id(df, random_neighbors)
    print("Random Neighbor Song IDs:", song_id_list) ##### !!!!! 이 부분에 song_id_list가 출력되는 게 아니라, song_id_list를 DB에 보내지게 바꿔줘야함.
    result = get_data_by_keys(df, song_id_list)
    return result

###### 실행부분
input_list = [6903, 8446, 3025, 801, 1998] ##### 인풋 : 임의로 지정했지만, 스프링부트에서 들어오는 인풋리스트로 변환. (input value - integer list --- "KEY")
output_json = run(input_list) ##### 아웃풋 :  type | list[dict] -> "json"형식으로 바꿔야 하면 변경해야함. (json.dump로 쏘면 될 것 같긴함.)
# print("Output:", output_json) ##### 확인용

Random Neighbor Song IDs: [7569, 1697, 8666, 82, 554]


In [4]:
loaded_model

KNeighborsClassifier(n_neighbors=10)

In [5]:
data={
  "song_ids": "[3348, 7534,3832,1456,3309]"
}

In [14]:
data['song_ids']

'[3348, 7534,3832,1456,3309]'

In [15]:
song_ids_str = data['song_ids']
song_ids_list = [int(x) for x in song_ids_str.strip('[]').split(',')]
print(song_ids_list) 

[3348, 7534, 3832, 1456, 3309]


In [17]:
song_ids_list

[3348, 7534, 3832, 1456, 3309]

In [25]:
def get_indices_from_song_ids(song_ids,df):
    indices = []
    for song_id in song_ids:
        try:
            index = df[df['key'] == song_id].index.values[0]
            indices.append(index)
        except IndexError:
            pass
    return indices

In [28]:
input_to_function=get_indices_from_song_ids(song_ids_list,df)

In [27]:
def get_random_neighbor_indices(index_list,X,loaded_model, k=10, random_samples=5):
    random_neighbors = set()
    for index in index_list:
        query_point = X[index].reshape(1, -1)
        indices = loaded_model.kneighbors(query_point, n_neighbors=k, return_distance=False)
        random_neighbors.update(indices[0][1:])

    # Convert the set to a list and shuffle
    random_neighbors_list = list(random_neighbors)
    np.random.shuffle(random_neighbors_list)

    # Take random samples (default: 5)
    random_neighbors_list = random_neighbors_list[:random_samples]

    return random_neighbors_list

In [40]:
random_neighbors = get_random_neighbor_indices(input_to_function,X,loaded_model, k=10,random_samples=5)
random_neighbors

[1177, 68, 1596, 301, 1293]

In [38]:
def indices_to_song_id(data_frame, indices_list):
    song_id_list = [data_frame.loc[index, 'key'] for index in indices_list]
    return song_id_list

In [41]:
song_id_list = indices_to_song_id(df, random_neighbors)
song_id_list

[6301, 4918, 1560, 315, 2554]

In [42]:
def get_data_by_keys(df, random_neighbors):
    data_list = []
    for index, row in df.iterrows():
        # Check if the 'key' column value is
        if row['key'] in random_neighbors:
            # Create a dictionary
            data_dict = {
                'key': row['key'],  ##### 만약에 key를 song_id로 전달해줘야하면 수정해야할 부분.
                'song_name': row['song_name'],
                'artist_name': row['artist_name'],
                'lyric_str': row['lyric_str'],
                'url_new_song': row['url_new_song']
            }

            # Add the dictionary to the list
            data_list.append(data_dict)
            # Convert the list of dictionaries to a JSON string
            json_data = json.dumps(data_list)

    return data_list

In [43]:
result = get_data_by_keys(df, song_id_list)
result

[{'key': 4918,
  'song_name': '또 사랑에 속다',
  'artist_name': '에이트',
  'lyric_str': '어차피 한 번은 겪을 이별 어차피 한 번은 흘릴 눈물 사랑은 속고 속이는 속고 속이는 전쟁이야 이별은 먼저 말하면 이기는 게임 그뿐이야 난 피지 못한 꽃 이곳은 온통 어둠 날 또 한 번 속인 것 그게 사랑이든 너든 쉽게 주지 않아 내게 필요한 양의 거름 딱 숨만 쉬어 그만큼만 양분을 주거든 남들에겐 사랑이 대단할지 몰라도 나에겐 별 볼일 없어 백이면 백 젖어 내 베갠 또다시 사랑이 날 속이고 익숙한 눈물이 흘러내려 아무리 잡아보려 행복하려 해도 내겐 이런 사랑만 오는 걸 또 사랑에 속고 또 이별에 울고 가슴이 와르르 무너지고 까맣게 점점 타 들어가도 억지로 웃고 거짓말을 하고 어차피 한번은 겪을 이별 어차피 한번은 흘릴 눈물 이렇게밖에 또 이렇게밖에 전에 겪었던 그리고 또 겪게 될 것 이별 없을 사랑은 결국 마지막 한번 그러니까 나머진 전부 다 결국 이별 그러니까 어차피 한 번은 겪을 이별 또다시 사랑이 날 속이고 익숙한 눈물이 흘러내려 아무리 잡아보려 행복하려 해도 내겐 이런 사랑만 오는 걸 또 사랑에 속고 또 이별에 울고 정말 이번만큼은 이러기 싫었는데 가슴이 와르르 무너지고 까맣게 점점 타 들어가도 억지로 바보같이 같은 말만 되뇌어 한번은 겪을 이별이라고 어차피 한 번은 겪을 이별 어차피 한 번은 흘릴 눈물 이 말이 다 거짓돼 핑계라고 해도 이렇게밖에 말 할 수 없어 어차피 한 번은 겪을 이별 어차피 한 번은 흘릴 눈물 사랑 은 속 고 속이는 속 고 속이는 전쟁 이야 이별 은 먼저 말 하면 이기는 게임 뿐 이야 난 피지 못 한 꽃 곳 은 온통 어둠 날 한 번 속인 그게 사랑 이든 든 쉽게 주지 않아 내게 필요한 양 거름 숨 만 쉬어 만큼만 양분 주거 든 남 에겐 사랑 대단할지 몰라도 에겐 별 볼일 없어 백이 면 백 젖어 내 베갠 또다시 사랑 날 속 이고 익숙한 눈물 흘러내려 아무리 잡아 보려 행복하려 해도 내겐